In [1]:
import markdown
from markdown_it import MarkdownIt
import json
import os
from rich.progress import track
import sys
sys.path.append('../')
from chatgpt_caller import chatgpt_caller
import numpy as np

In [2]:
file_path = '../../input/subgraph/subgraph.md'
CONTEXT_TOKEN_LIMIT = 4000

In [3]:
file_name = file_path.split('/')[-1].split('.')[0]
output_path = f"../../output/{file_name}/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [4]:
def markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        markdown_text = file.read()

    # 使用markdown-it解析Markdown文本
    md = MarkdownIt()
    tokens = md.parse(markdown_text, {})
    return tokens

In [5]:
parsed_markdown = markdown(file_path)

In [6]:
# children can include tree node or text node
# tree_node = {'node_type': 0, 'level': 0, 'title': '', 'children': []}
# text_node = {'node_type': 1, 'text': ''}

In [7]:
def tokens2node(tokens):
  all_nodes = []
  for i, token in enumerate(tokens):
    if token.type == 'heading_open':
      level = int(token.tag[1])
      node = {'node_type': 0, 'level': level, 'text': tokens[i+1].content, 'children': []}
      all_nodes.append(node)
    elif token.type == 'paragraph_open':
      node = {'node_type': 1, 'text': tokens[i+1].content}
      all_nodes.append(node)
  return all_nodes

In [8]:
all_nodes = tokens2node(parsed_markdown)

In [9]:
def clean_title_level(all_nodes):
  for node in all_nodes:
    if node['node_type'] == 0:
      title_words = node['text'].split(" ")

      first_word = ""
      for i in title_words:
        if i != "":
          first_word = i
          break

      nums = first_word.split(".")
      is_num = True
      for i in nums:
        if not i.isdigit():
          is_num = False

      if is_num:
        node['level'] = len(nums) + 1
    
  return all_nodes

In [10]:
cleaned_all_nodes = clean_title_level(all_nodes)

In [11]:
def build_tree(root_node, nodes, start_idx, current_level):
    idx = start_idx
    while idx < len(nodes):
        node = nodes[idx]
        if node['node_type'] == 1:
            root_node['children'].append(node)
            idx += 1
        else:
            if node['level'] <= current_level:
                break
            else:
                root_copy = node.copy()
                child_tree, idx = build_tree(root_copy, nodes, idx + 1, current_level=node['level'])
                root_node['children'].append(child_tree)
    return root_node, idx

def organize_tokens(nodes):
    root = nodes[0]
    root_tree, _ = build_tree(root, nodes, 1, 1)
    return root_tree

In [12]:
file_tree = organize_tokens(cleaned_all_nodes)

In [13]:
file_tree

{'node_type': 0,
 'level': 1,
 'text': 'Subgraph Search Over Neural-Symbolic Graphs',
 'children': [{'node_type': 1,
   'text': 'Ye Yuan Beijing Institute of Technology yuan-ye@bit.edu.cn Anbiao Wu Northeastern University, China anbiaowu@stumail.neu.edu.cn'},
  {'node_type': 0,
   'level': 2,
   'text': 'Abstract',
   'children': [{'node_type': 1,
     'text': 'In this paper, we propose *neural-symbolic graph databases* (NSGDs) that extends traditional graph data with content and structural embeddings in every node. The content embeddings can represent unstructured data (e.g., images, videos, and texts), while structural embeddings can be used to deal with incomplete graphs. We can advocate machine learning models (e.g., deep learning) to transform unstructured data and graph nodes to these embeddings. NSGDs can support a wide range of applications (e.g., online recommendation and natural language question answering) in social-media networks, multi-modal knowledge graphs and etc. As a 

In [14]:
def node2text(node):
  context = node['text'] + '\n'
  if node['node_type'] == 1:
    return context
  if node['node_type'] == 0:
    for n in node['children']:
      context += node2text(n)
  return context

In [15]:
def file2text_blocks(file_tree):
  return extract_text_blockes(file_tree)

def extract_text_blockes(node):
  text_blocks = []
  if node['node_type'] == 1:
    text_blocks.append(node2text(node))
  if node['node_type'] == 0:
    for n in node['children']:
      text_blocks += extract_text_blockes(n)
  return text_blocks

In [16]:
text_blocks = file2text_blocks(file_tree)

In [17]:
text_blocks

['Ye Yuan Beijing Institute of Technology yuan-ye@bit.edu.cn Anbiao Wu Northeastern University, China anbiaowu@stumail.neu.edu.cn\n',
 'In this paper, we propose *neural-symbolic graph databases* (NSGDs) that extends traditional graph data with content and structural embeddings in every node. The content embeddings can represent unstructured data (e.g., images, videos, and texts), while structural embeddings can be used to deal with incomplete graphs. We can advocate machine learning models (e.g., deep learning) to transform unstructured data and graph nodes to these embeddings. NSGDs can support a wide range of applications (e.g., online recommendation and natural language question answering) in social-media networks, multi-modal knowledge graphs and etc. As a typical search over graphs, we study subgraph search over a large NSGD, called *neural-symbolic subgraph matching* (NSMatch) that includes a novel ranking search function. Specifically, we develop a general algorithmic framework

In [18]:
gpt_caller = chatgpt_caller(output_path, 'EMPTY')

In [19]:
def add_embeding2node(node, parent_title): 
  all_texts = []
  all_embedings = [] 
  if node['node_type'] == 1:
    node['embedding'] = gpt_caller.get_embedding(node['text'])
    all_texts.append({"text": node['text'], "title": parent_title})
    all_embedings.append(node['embedding'])
  if node['node_type'] == 0:
    for n in node['children']:
      texts, embedings = add_embeding2node(n, node['text'])
      all_texts += texts
      all_embedings += embedings
  return all_texts, all_embedings

def add_embeding2file_tree(file_tree):
  return add_embeding2node(file_tree, file_tree['text'])

In [20]:
all_texts, all_embedings = add_embeding2file_tree(file_tree)

In [21]:
all_texts

[{'text': 'Ye Yuan Beijing Institute of Technology yuan-ye@bit.edu.cn Anbiao Wu Northeastern University, China anbiaowu@stumail.neu.edu.cn',
  'title': 'Subgraph Search Over Neural-Symbolic Graphs'},
 {'text': 'In this paper, we propose *neural-symbolic graph databases* (NSGDs) that extends traditional graph data with content and structural embeddings in every node. The content embeddings can represent unstructured data (e.g., images, videos, and texts), while structural embeddings can be used to deal with incomplete graphs. We can advocate machine learning models (e.g., deep learning) to transform unstructured data and graph nodes to these embeddings. NSGDs can support a wide range of applications (e.g., online recommendation and natural language question answering) in social-media networks, multi-modal knowledge graphs and etc. As a typical search over graphs, we study subgraph search over a large NSGD, called *neural-symbolic subgraph matching* (NSMatch) that includes a novel rankin

In [22]:
def print_tree_titles(node):
  print(node['level']*"#" + " " + node['text'])

  for n in node['children']:
    if n['node_type'] == 0:
      print_tree_titles(n)

In [23]:
print_tree_titles(file_tree)

# Subgraph Search Over Neural-Symbolic Graphs
## Abstract
## Ccs Concepts - Information Systems → **Graph-Based Database Models**. Keywords
## 1 Introduction
## 2 Problem Definition
## 3 Framework Of Search
## 4 Top-K Star Matching
### 4.2 Top-H Node Matching
### 4.3 Edge Judging
## (1) Symbolic Enhancement.
## 5 Top-K Join & Pattern Decomposition
### 5.1 Top-K Join
### 5.2 Pattern Decomposition
## 6 Evaluation
### 6.1 Setup
### 6.2 Experimental Results
## 7 Related Work
## 8 Conclusion
## Acknowledgments
## References


In [24]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query, all_embedings):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = gpt_caller.get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in enumerate(all_embedings)
    ], reverse=True, key=lambda x: x[0])
    
    return document_similarities

def get_prompt(question, all_texts, all_embedings):
    ordered_candidates = order_document_sections_by_query_similarity(question, all_embedings)
    ctx = u""
    for candi in ordered_candidates:
        next = ctx + u"\n" + "- from section " + all_texts[candi[1]]['title'] + " : " + all_texts[candi[1]]['text']
        if len(next)> CONTEXT_TOKEN_LIMIT:
            break
        ctx = next
    if len(ctx) == 0:
        return u""    
    
    prompt = u"".join([
        u"Answer the question base on the context, answer in the same language of question\n\n"
        u"Context:" + ctx + u"\n\n"
        u"Question:" + question + u"\n\n"
    ])

    return prompt

In [25]:
questions = ["What is the main idea of this paper?",
             "What is the definition of the problem?",
             "What are the difficulties of the problem?",
             "What the papers in the past do on the problem?",
             "What downsides the past papers have?",
             "Summarize the main steps of the method and the advantage of the steps."]

In [26]:
# for question in questions:
#   print("question: ", question)
#   prompt = get_prompt(question, all_texts, all_embedings)
#   print("prompt:", prompt)
#   res = gpt_caller.chat(prompt=prompt)
#   print("answer: ", res.content)

In [27]:
def node2text_with_summary(node):
  if node['node_type'] == 1:
    return
  
  context = node['text'] + '\n'
  for n in node['children']:
    if n['node_type'] == 1:
      context += n['text'] + '\n'
    if n['node_type'] == 0:
      context += n["text"]
      context += "The summary of the section: "
      context += n['summary_text']
  return context

In [28]:
def summary_tree(file_tree):
  return summary_node(file_tree)

def get_prompt_summary_text(text):
  prompt = u"".join([
    u"Summary the following text with one to two sentences:\n\n"
    u"text: " + text + u"\n\n"
    u"summary: The text discusses " + u"\n\n"
  ])

  return prompt

def get_summary(prompt):
  res = gpt_caller.chat(prompt=prompt)
  # TODO: add check summary logic (1-2 sentences, without useless repitition)
  return res.content

def summary_node(node):
  if node['node_type'] == 1:
    return
  if "summary_text" in node.keys() and "summary_embeding" in node.keys():
    return
  
  print("summary_node: ", node['text'])

  all_text_in_node = node2text(node)
  prompt = get_prompt_summary_text(all_text_in_node)
  if len(prompt) < CONTEXT_TOKEN_LIMIT:
    node["summary_text"] = get_summary(prompt)
    node["summary_embeding"] = gpt_caller.get_embedding(node["summary_text"])
  else:
    # If the text is too long,
    # first finish summary all children title node 
    for n in node['children']:
      summary_node(n)

    text_in_node_with_summary = node2text_with_summary(node)
    prompt = get_prompt_summary_text(text_in_node_with_summary)
    if len(prompt) < CONTEXT_TOKEN_LIMIT:
      print("The section is two large to be summarized.")
      print("Prompt: ", prompt)
      node["summary_text"] = ""
      node["summary_embeding"] = []
      return
    node["summary_text"] = get_summary(prompt)
    node["summary_embeding"] = gpt_caller.get_embedding(node["summary_text"])

In [29]:
# summary_tree(file_tree)

In [30]:
def print_tree_titles_summary(node):
  print(node['level']*"#" + " " + node['text'])
  print("section summary: ", node["summary_text"])

  for n in node['children']:
    if n['node_type'] == 0:
      print_tree_titles_summary(n)

In [31]:
# print_tree_titles_summary(file_tree)

In [32]:
sections = [
  "1 Introduction",
  "2 Problem Definition",
  "3 Framework Of Search",
  "4 Top-K Star Matching",
  "5 Top-K Join & Pattern Decomposition",
  "6 Evaluation",
  "7 Related Work",
  "8 Conclusion",
  "Acknowledgments",
  "References"
]
section_summaries = [
  "The text discusses the use of graphs to represent real-life data, with an emphasis on multi-modal graphs that contain both structured and unstructured data. The authors propose a new graph data model called the neural-symbolic graph database (NSGD) that extends property graph models with content and structural embeddings in every node and edge. They also introduce a novel algorithm called NSMatch for subgraph search over NSGDs, which uses sophisticated ranking functions to generate top answers in a monotonic decreasing order of matching score. The authors provide experimental results that demonstrate the effectiveness and efficiency of their algorithms.",
  "The text discusses the problem of Neural-Symbolic Subgraph Matching (NSMatch) in Neural-Symbolic Graph Databases (NSGDs). NSGDs are directed labeled graphs that consist of nodes, edges, labels, attributes, and vectors representing unstructured data. NSMatch aims to find a subgraph match in an NSGD based on a given graph pattern and a similarity function. The problem is to find the top-k subgraph matches in the NSGD that meet a certain matching score. NSMatch is NP-hard due to its special case being NP-complete.",
  "The text discusses an algorithmic framework for attacking the hard problem of NSMatch. The framework consists of three main steps: pattern decomposition, star matching, and top-k join.",
  "The text discusses the SMat algorithm for computing top-k star matches in a star pattern. The algorithm consists of three phases: Phase 1 identifies candidate node matches and top-h leaf matches; Phase 2 selects the best k star matches to form a pseudo top-k star matches and sorts the leaf matches; and Phase 3 maintains the priority queue and pops the best match until the desired top-k matches are obtained. The algorithm uses the content vectors Csq and Cg, as well as the structural vector Sg, to identify edges and compute the similarity function δ(·) between Csq and Cg. The value of h, which determines the number of leaf matches to be scanned, is pre-determined in the system. The algorithm is efficient and can accurately complete the missing edges of top-k star matches.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         5 Top-K Join & Pattern Decomposition",
  "",
  "",
  "The text discusses the related work on subgraph search and approximate nearest neighbors search (ANNS) for graph-based methods. It categorizes the subgraph search methods into symbolic methods and neural approaches, and provides examples of each category. It also focuses on graph-based ANNS methods, particularly HNSW and NSG, and explains how NSMatch differs from traditional subgraph matching and existing ANNS algorithms.",
  "The text discusses a new method called neural-symbolic subgraph matching (NSMatch) that uses a neuralsymbolic graph database (NSGD) model to support applications of multi-modal knowledge graphs and multi-modal social medias. The method includes a general and efficient algorithmic framework to process NSMatch, strategies of edge deletion to speed up graph-based ANNS, and a neural-symbolic learning model to complete the missing edges of the NSGD. The text also mentions future work, such as studying incremental NSMatch and investigating more pattern types over large NSGDs.",
  "The text acknowledges the financial support provided by various organizations to the authors Ye Yuan and Jianbin Qin for their research on the National Key R&D Program of China, the NSFC, and the DITDP.",
  "The text discusses various approaches and techniques for efficient and accurate subgraph matching, similarity search, and knowledge graph completion. It covers techniques such as using approximate nearest neighbor queries, translating embeddings, and graph-based methods. The paper also discusses the use of knowledge graphs for text-centric information retrieval and efficient k-nearest neighbor graph construction."
]


In [33]:
for question in questions:
  print("##################")
  res = gpt_caller.choose(f"You are a skillful paper reader. For answering the question: {question}, you need to choose which sections to read.", sections, choices_explain=section_summaries, verbose=True)
  print("### question: ", question)
  print("### res: ", res)

##################


System prompt: You are a chooser returning indexes of choices according to the stem.
             You are to output the indexes in the format: {"indexes": [number, number, number]}.
             For example:
             The stem: choose from the choices.
             Choices:
             0 @@cat@@
             Explaination of the choice: This is a cat
             1 @@dog@@
             Explaination of the choice: This is a dog
             2 @@apple@@
             Explaination of the choice: This is an apple
             3 @@banana@@
             Explaination of the choice: This is a banana
             Your Answer: {"indexes": [0, 3]}
             Just Return the Answer and Don't Explain the Answer!!!


User prompt: The stem: You are a skillful paper reader. For answering the question: What is the main idea of this paper?, you need to choose which sections to read.
Choices:
0  @@1 Introduction@@
Explanation of the choice: The text discusses the use of graphs to represent real-life 